In [ ]:
#re enable if not downloaded
#!pip install soundcard
#!pip install keyboard
#!pip install demucs #needed later

In [ ]:

import soundcard as sc
import soundfile as sf
import keyboard
import numpy as np
import warnings

# Suppress specific warnings related to data discontinuity
warnings.filterwarnings("ignore", category=sc.SoundcardRuntimeWarning)

# Output filenames
mic_output_file = "microphone_output.wav"
sys_output_file = "system_output.wav"

SAMPLE_RATE = 48000  # [Hz], sampling rate.
chunk_size = 4096    # Increased size to reduce handling frequency

# Buffers to store the recorded data
mic_frames = []
sys_frames = []

print("Press SPACE to start recording microphone and system audio...")
keyboard.wait('space')  # Wait for the first press of spacebar to start recording
print("Recording... Press SPACE again to stop.")

# Get the default microphone and default speaker (loopback for system audio)
microphone = sc.default_microphone()
speaker = sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True)

# Open both the microphone and system audio streams
with microphone.recorder(samplerate=SAMPLE_RATE, blocksize=chunk_size) as mic_recorder, \
     speaker.recorder(samplerate=SAMPLE_RATE, blocksize=chunk_size) as sys_recorder:

    while True:
        try:
            # Record a chunk of data from both microphone and system audio
            mic_data = mic_recorder.record(numframes=chunk_size)
            sys_data = sys_recorder.record(numframes=chunk_size)

            # Append the recorded chunks to their respective buffers
            mic_frames.append(mic_data)
            sys_frames.append(sys_data)

        except sc.SoundcardRuntimeWarning as e:
            print(f"Warning caught: {e}")
            continue  # Ignore the warning and continue recording

        # Check if the spacebar is pressed again to stop the recording
        if keyboard.is_pressed('space'):
            print("Stopping recording...")
            break

# Convert the recorded frames to a NumPy array
mic_recorded_data = np.concatenate(mic_frames, axis=0)
sys_recorded_data = np.concatenate(sys_frames, axis=0)

# Save the recorded data to output files
sf.write(file=mic_output_file, data=mic_recorded_data[:, 0], samplerate=SAMPLE_RATE)
sf.write(file=sys_output_file, data=sys_recorded_data[:, 0], samplerate=SAMPLE_RATE)

print(f"Microphone recording saved to {mic_output_file}")
print(f"System audio recording saved to {sys_output_file}")
 